Грузим либы

In [1]:
from glob import glob
import pandas as pd
from collections import Counter
import math
import itertools
from difflib import SequenceMatcher

Две ключевых метрики рассчитываются с помощью:
    - Функции для расчёта схожести двух списков `counter_cosine_similarity`
    - Метода для расчёта схожести двух документов `SequenceMatcher.ratio()`

In [2]:
def counter_cosine_similarity(c1, c2):
    '''
    Функция для расчёта схожести двух списков
    '''
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

Читаем данные из студенческих файлов с кодом. Файлы хранятся в папке `works` с разрешением `py`

In [3]:
files = glob('works/*.py')
all_data = []
for file in files:
    file_data = []
    with open(file, 'r', encoding='utf-8') as f:
        file_data = f.readlines()
    d = {'name':file.replace('works\\review_homework_id_',''), 'file_data':file_data}
    all_data.append(d)

Предобратываем данные:
- убираем строки, начинающиеся с #;
- удаляем пустые строки;
- убираем in-line комментарии
- убираем multiline комментарии

In [4]:
for item in all_data:
    rows = item['file_data']
    new_rows = []
    kr = False
    for row in rows:
        row = row.strip()
        if row.startswith('#'):
            continue
        if row == '\n':
            continue
        if len(row) == 0:
            continue
        if '#' in row:
            row = row.split('#')[0]
        if (row == '"""') and (kr == False):
            kr = True
            continue
        if (row == '"""') and (kr == True):
            kr = False
            continue
        if (row != '"""') and (kr == True):
            continue
        row = row.strip()
        new_rows.append(row)
    item['new_file_data'] = new_rows

Считаем статистику схожести по парам

In [5]:
combs = set(itertools.combinations([item['name'] for item in all_data], 2))
stats = []
for comb in combs:
    a1 = next(item for item in all_data if item["name"] == comb[0])
    a2 = next(item for item in all_data if item["name"] == comb[1])
    counterA = Counter(a1['new_file_data'])
    counterB = Counter(a2['new_file_data'])
    similarity = counter_cosine_similarity(counterA,counterB)
    similarity_2 = SequenceMatcher(None, a1['new_file_data'], a2['new_file_data']).ratio()
    seq_match_blocks = SequenceMatcher(None, a1['new_file_data'], a2['new_file_data']).get_matching_blocks()
    d = {'doc1':comb[0],'doc2':comb[1],'similarity':similarity, 'similarity_2':similarity_2, 'matching_blocks': seq_match_blocks}
    stats.append(d)

Загоняем статистику в датафрейм

In [6]:
df = pd.DataFrame(stats).sort_values('similarity',ascending=False)
display(df.head())

,doc1,doc2,similarity,similarity_2,matching_blocks
95,124111.py,124350.py,0.581055,0.380048,"[(0, 0, 1), (4, 1, 1), (6, 3, 1), (7, 21, 1), ..."
100,125915.py,127334.py,0.276196,0.155172,"[(0, 0, 1), (1, 3, 1), (2, 39, 1), (3, 53, 1),..."
0,124687.py,125378.py,0.275522,0.112245,"[(0, 0, 2), (2, 46, 1), (29, 47, 4), (52, 51, ..."
97,124408.py,127942.py,0.239011,0.156250,"[(1, 0, 2), (3, 3, 1), (15, 32, 2), (17, 35, 1..."
101,125378.py,125915.py,0.238422,0.149068,"[(0, 0, 2), (3, 3, 1), (5, 4, 1), (6, 6, 1), (..."


Проанализируем самый совпадающий блок кода

In [7]:
index = 0
matching_blocks = sorted(df.iloc[index]['matching_blocks'], key=lambda x: x.size, reverse=True)
print(matching_blocks[0])
print(' ')
doc1 = next(item for item in all_data if item["name"] == df.iloc[index]['doc1'])['new_file_data']
doc2 = next(item for item in all_data if item["name"] == df.iloc[index]['doc2'])['new_file_data']
print('doc1')
print(doc1[matching_blocks[0].a:matching_blocks[0].a+matching_blocks[0].size])
print('')
print('doc2')
print(doc2[matching_blocks[0].b:matching_blocks[0].b+matching_blocks[0].size])

Match(a=39, b=45, size=43)
 
doc1
['l = [', "{ 'A': 1, 'B': 3 },", "{ 'A': 2, 'B': 3 },", "{ 'A': 4, 'B': 5 }", ']', 'dfq = pd.DataFrame(l)', 'dfq', 'def squared(df):', "'''", 'Функция принимает таблицу, меняет элементы на их квадраты и возвращает уже таблицу с квадратами.', "'''", "df['A'] = df['A']*df['A']", "df['B'] = df['B']*df['B']", 'return df', 'squared(dfq)', 'dfq', 'def unsquared(df):', "'''", 'Берет квадратный корень.', 'Принимает: таблицу.', 'Возвращает: таблицу, из элементов которой взят квадратный корень.', "'''", 'import numpy as np', "df['A'] = np.sqrt(df['A']).astype(int)", "df['B'] = np.sqrt(df['B']).astype(int)", 'return df', 'unsquared(dfq)', 'dfq', 'def sum_(row):', "'''", 'Функция принимает таблицу построчно. Возвращает сумму элементов.', 'Применяется ко всей таблице через метод apply с аргументом axis=1, то есть построчно.', "'''", "return row['A']+row['B']", "dfq['Sum'] = dfq.apply(sum_, axis=1)", 'dfq', "dfq['Sum2'] = dfq.apply(lambda row: row['A'] + row['B'], a